### 使用大语言模型和 Langchain 实现一个客服机器人

客服机器人的场景，包含以下功能：
1. 该机器人能根据用户的输入判断用户要咨询的问题类型，进行自动调用相应的方法，进行返回。
2. 能处理的问题类型有：
    * 预定机票
    * 推荐商品
    * 订单查询
    * 一般购买咨询
3. 要能够实现多轮对话，例如预定机票的场景下，用户提出要订机票，机器人询问要订的日期和航班信息，用户提供信息。如果用户提供的信息不完全，则会再次询问，直到获取了所有的信息，能够完成预定。

在这部分，我们使用 LangChain 的 *RouterChain* 来实现多场景客服。

LangChain 提供了两种 *RouterChain*，它是使用语言模型通过语义理解，根据用户输入判断用户的意图，然后再结合多个不同的Chain对象，不同的意图使用不同的Chain进行处理。

有两种类型的*RouterChain*， 一种是 *LLMRouterChain*，一种是 *EmbeddingRouterChain*，顾名思义，一个是使用大语言模型进行意图识别，一种是使用 Embedding模型。使用 Embedding模型可以提供比较好的性价比。

首先还是先创建 SagemakerEndpoint 类型的model。

In [22]:
from typing import Dict
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler

import json

import logging
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input = {"ask": prompt, **model_kwargs}
        logger.info("prompt: %s", prompt)
        logger.info("model_kwargs: %s", model_kwargs)
        input_str = json.dumps(input)
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        logger.info("response_json: %s", response_json)
        return response_json["answer"]

content_handler = ContentHandler()

alpaca_model = SagemakerEndpoint(
    endpoint_name="chinese-alpaca-plus-7b",
    region_name="us-east-1", 
    model_kwargs={"temperature": 0.001, "top_p": 0.3},
    content_handler=content_handler
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


对于客服和售前咨询的场景，我们可以使用定制的Prompt和Chain，来进行基于知识库的问答。

对于商品推荐，我们将商品信息embedding后，保存到向量数据库中，推荐的时候，从该向量库中查询最相近的5个产品，然后让语言模型进行回复。

对于售前咨询，也是将售前咨询信息，以知识库的形式进行保存，然后进行基于知识库的问答。

In [23]:
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate



recommend_template = """你是一个机器人客服，需要给客户推荐商品，根据下面搜索得到的商品信息，给用户提供推荐.

商品信息:


### Question:
{question}

### Response:
"""

faq_template = """你是一个机器人客服, 使用下面的已知内容，简洁、准确的回答最后的问题，并使用中文。如果你不知道答案，尽量让用户提供更多的信息，不要编造答案。

已知内容:


问题: {question}
答案:"""

recommend_prompt = PromptTemplate(template=recommend_template, input_variables=["question"])
recommend_chain = LLMChain(llm=alpaca_model, prompt=recommend_prompt)

faq_prompt = PromptTemplate(template=faq_template, input_variables=["question"])
faq_chain = LLMChain(llm=alpaca_model, prompt=faq_prompt)


我们本来是要使用基于知识库的搜索，但是之前已经演示过加载知识库，这里为了方便，就使用普通的Chain，进行问答。

In [24]:
recommend_chain({"question": "想买一件西服"})

INFO:root:prompt: 你是一个机器人客服，需要给客户推荐商品，根据下面搜索得到的商品信息，给用户提供推荐.

商品信息:


### Question:
想买一件西服

### Response:

INFO:root:model_kwargs: {'temperature': 0.001, 'top_p': 0.3}
INFO:root:response_json: {'answer': '您可以考虑购买这款品牌的西装：[link]。'}


{'question': '想买一件西服', 'text': '您可以考虑购买这款品牌的西装：[link]。'}

In [25]:
faq_chain({"question": "上次买的西服想退货"})

INFO:root:prompt: 你是一个机器人客服, 使用下面的已知内容，简洁、准确的回答最后的问题，并使用中文。如果你不知道答案，尽量让用户提供更多的信息，不要编造答案。

已知内容:


问题: 上次买的西服想退货
答案:
INFO:root:model_kwargs: {'temperature': 0.001, 'top_p': 0.3}
INFO:root:response_json: {'answer': '你是一个机器人客服, 使用下面的已知内容，简洁、准确的回答最后的问题，并使用中文。如果你不知道答案，尽量让用户提供更多的信息，不要编造答案。\n\n已知内容:\n\n\n问题: 上次买的西服想退货\n答案: 很抱歉，我们公司不支持商品的退换货服务。'}


{'question': '上次买的西服想退货',
 'text': '你是一个机器人客服, 使用下面的已知内容，简洁、准确的回答最后的问题，并使用中文。如果你不知道答案，尽量让用户提供更多的信息，不要编造答案。\n\n已知内容:\n\n\n问题: 上次买的西服想退货\n答案: 很抱歉，我们公司不支持商品的退换货服务。'}

#### 方法调用的Chain
在上一个实例中，我们使用Embedding识别用户意图，再对应到相应的function，然后识别函数的参数，然后进行调用，返回结果。

这一次，我们使用 LangChain 的 *TransformChain* 来实现函数调用的问题。

TransformChain 本来是用于通过一个函数，将输入的参数，通过函数调用转换成另一个结果。使用这个Chain，我们可以调用我们的业务方法，来实现语言模型调用业务方法。

In [36]:
import datetime, re, ast
from langchain.chains import TransformChain

import logging
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)

def search_order(inputs: dict) -> dict:
    logger.info("==== search_order inputs:" + str(inputs))
    inputs = inputs.get("params")

    if type(inputs) == str:
        inputs = ast.literal_eval(inputs)
    if type(inputs) == list:
        inputs = inputs[0]
    order_provided = inputs.get("order_number")
    if not order_provided:
        result = "请问您的订单号是多少？"
    else:
        pattern = r"\d+[A-Z]+"
        match = re.search(pattern, order_provided)

        result = ""
        order_number = order_provided
        if match:
            order_number = match.group(0)
            result = "订单: " + order_number + "状态：已发货；发货日期：2023-05-01；预计送达时间：2023-05-10"
        else:
            result = "提供的订单号: " + order_number +" 不存在."
    return {"text": result}


def order_flight(inputs: dict) -> str:
    logger.info("==== search_order inputs:" + str(inputs))
    inputs = inputs.get("params")
    if type(inputs) == str:
        inputs = ast.literal_eval(inputs)
    if type(inputs) == list:
        inputs = inputs[0]
        
    result = ""
    flight_date = inputs.get("flight_date")
    if type(flight_date) == list:
        flight_date = flight_date[0]
        
    flight_no = inputs.get("flight_no")
    if type(flight_no) == list:
        flight_no = flight_no[0]
        
    if not flight_date and not flight_no:
        result = "请提供出行日期和预定航班"
    elif not flight_date:
        result = "请提供出行日期"
    elif not flight_no:
        result = "请提供航班号"
    result = "预定：" + flight_date + ", 航班号：" + flight_no
    return {"text": result}


order_search_chain = TransformChain(input_variables=["params"], output_variables=["text"], transform=search_order)

order_flight_chain = TransformChain(input_variables=["params"], output_variables=["text"], transform=order_flight)


这里定义了两个TransformChain。以及两个函数，分别用于查询用户订单，和为用户预订机票。

可以看到，这两个函数都不同的参数，参数的获取也要从inputs中获取，inputs的格式下面会讲到。

In [37]:
# 尝试调用查询订单的 chain
order_search_chain({"params": {"order_number": "我的订单没有收到 2021ABC"}})

INFO:root:==== search_order inputs:{'params': {'order_number': '我的订单没有收到 2021ABC'}}


{'params': {'order_number': '我的订单没有收到 2021ABC'},
 'text': '订单: 2021ABC状态：已发货；发货日期：2023-05-01；预计送达时间：2023-05-10'}

有了函数，我们还需要从用户的输入里面提取函数所需的参数，所以，我们定义了另一个Chain，用于提取参数信息。

In [38]:
from langchain import PromptTemplate, LLMChain

prompt_template = (
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction: 从以下对输入中信息提取'{input}', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 找不到所需信息则返回空, 结果用json格式."
    "### Input: {question} \n\n"
    "### Response:\n"
)
prompt = PromptTemplate(
    input_variables=["question", "input"],
    template=prompt_template
)

# func_inputs 里面是每个function所需要的参数列表
func_inputs = {"search_order": ['order_number'], "order_flight": ['flight_date', 'flight_no']}

extract_chain = LLMChain(llm=alpaca_model, prompt=prompt, output_key="params")

In [39]:
# 调用 chain，提取参数
extract_chain({"question": "我的订单没有收到, 订单号是 2023ABC", "input": func_inputs.get("search_order")})

INFO:root:prompt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 从以下对输入中信息提取'['order_number']', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 找不到所需信息则返回空, 结果用json格式.### Input: 我的订单没有收到, 订单号是 2023ABC 

### Response:

INFO:root:model_kwargs: {'temperature': 0.001, 'top_p': 0.3}
INFO:root:response_json: {'answer': '[{"order_number": "2023ABC"}]'}


{'question': '我的订单没有收到, 订单号是 2023ABC',
 'input': ['order_number'],
 'params': '[{"order_number": "2023ABC"}]'}

In [40]:
extract_chain({"question": "想要预定今天到上海的CA1234的航班", "input": func_inputs.get("order_flight")})

INFO:root:prompt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 从以下对输入中信息提取'['flight_date', 'flight_no']', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 找不到所需信息则返回空, 结果用json格式.### Input: 想要预定今天到上海的CA1234的航班 

### Response:

INFO:root:model_kwargs: {'temperature': 0.001, 'top_p': 0.3}
INFO:root:response_json: {'answer': '[{"flight_date": "2021-08-22", "flight_no": "CA1234"} ]'}


{'question': '想要预定今天到上海的CA1234的航班',
 'input': ['flight_date', 'flight_no'],
 'params': '[{"flight_date": "2021-08-22", "flight_no": "CA1234"} ]'}

可以看到，不同的输入，都能够提取到参数，但是，提取的结果，虽然都是json格式的，但是航班的参数提取，放在了list里。

所以，在对应的上面的 *order_flight* 和 *order_search* 方法里，我们都对输入 inputs 做了一些处理，保证在各种情况下，都能够得到参数值。

```python
inputs = inputs.get("params")
if type(inputs) == str:
    inputs = json.loads(inputs)
if type(inputs) == list:
    inputs = inputs[0]

result = ""
flight_date = inputs.get("flight_date")
if type(flight_date) == list:
    flight_date = flight_date[0]

flight_no = inputs.get("flight_no")
if type(flight_no) == list:
    flight_no = flight_no[0]
```

由于我们的chain最终只接收用户问题，所以，我们需要能得到各个函数的参数列表。

In [41]:
# func_inputs = {"search_order": ['order_number'], "order_flight": ['flight_date', 'flight_no']}

def get_order_func_inputs(inputs: dict) -> list:
    logger.info("get_order_func_inputs:" + str(inputs))
    return {"input": func_inputs.get("search_order")}
order_search_param_chain = TransformChain(input_variables=["question"], output_variables=["input"], transform=get_order_func_inputs)

def get_flight_func_inputs(inputs: dict) -> list:
    logger.info("get_flight_func_inputs:" + str(inputs))
    return {"input": func_inputs.get("order_flight")}
order_flight_param_chain = TransformChain(input_variables=["question"], output_variables=["input"], transform=get_flight_func_inputs)


然后，就用 SequentialChain 把信息提取的chain和调用function的chain连起来。

例如调用 order_search_chain_seq 的时候，调用流程如下：
1. 先调用一个chain，调用方法 get_order_func_inputs，获取查询订单这个方法所需的参数。
2. 再调用extract_chain提取方法参数。这时候，输入参数有 “question”，以及上一步生成的 “input”
3. 再用这两个参数调用 order_search_chain，该chain对调用order_search 方法。

In [43]:
from langchain.chains import SequentialChain

order_search_chain_seq = SequentialChain(
    chains=[order_search_param_chain, extract_chain, order_search_chain], input_variables=["question"],
    output_variables=["text"], verbose=True
)

order_flight_chain_seq = SequentialChain(
    chains=[order_flight_param_chain, extract_chain, order_flight_chain], input_variables=["question"],
    output_variables=["text"], verbose=True
)

测试运行

In [44]:
order_search_chain_seq({"question": "我的订单一直没收到，订单是 2023ABC"})

INFO:root:get_order_func_inputs:{'question': '我的订单一直没收到，订单是 2023ABC'}
INFO:root:prompt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 从以下对输入中信息提取'['order_number']', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 找不到所需信息则返回空, 结果用json格式.### Input: 我的订单一直没收到，订单是 2023ABC 

### Response:

INFO:root:model_kwargs: {'temperature': 0.001, 'top_p': 0.3}




> Entering new SequentialChain chain...


INFO:root:response_json: {'answer': '[{"order_number": "2023ABC"}]'}
INFO:root:==== search_order inputs:{'question': '我的订单一直没收到，订单是 2023ABC', 'input': ['order_number'], 'params': '[{"order_number": "2023ABC"}]'}



> Finished chain.


{'question': '我的订单一直没收到，订单是 2023ABC',
 'text': '订单: 2023ABC状态：已发货；发货日期：2023-05-01；预计送达时间：2023-05-10'}

In [45]:
order_search_chain_seq({"question": "我的订单一直没收到"})

INFO:root:get_order_func_inputs:{'question': '我的订单一直没收到'}
INFO:root:prompt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 从以下对输入中信息提取'['order_number']', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 找不到所需信息则返回空, 结果用json格式.### Input: 我的订单一直没收到 

### Response:

INFO:root:model_kwargs: {'temperature': 0.001, 'top_p': 0.3}




> Entering new SequentialChain chain...


INFO:root:response_json: {'answer': '[{"order_number": ""}]'}
INFO:root:==== search_order inputs:{'question': '我的订单一直没收到', 'input': ['order_number'], 'params': '[{"order_number": ""}]'}



> Finished chain.


{'question': '我的订单一直没收到', 'text': '请问您的订单号是多少？'}

In [46]:
order_flight_chain_seq({"question": "想要预定今天到上海的CA1234的航班"})

INFO:root:get_flight_func_inputs:{'question': '想要预定今天到上海的CA1234的航班'}
INFO:root:prompt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 从以下对输入中信息提取'['flight_date', 'flight_no']', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 找不到所需信息则返回空, 结果用json格式.### Input: 想要预定今天到上海的CA1234的航班 

### Response:

INFO:root:model_kwargs: {'temperature': 0.001, 'top_p': 0.3}




> Entering new SequentialChain chain...


INFO:root:response_json: {'answer': '[{"flight_date": "2021-08-22", "flight_no": "CA1234"} ]'}
INFO:root:==== search_order inputs:{'question': '想要预定今天到上海的CA1234的航班', 'input': ['flight_date', 'flight_no'], 'params': '[{"flight_date": "2021-08-22", "flight_no": "CA1234"} ]'}



> Finished chain.


{'question': '想要预定今天到上海的CA1234的航班', 'text': '预定：2021-08-22, 航班号：CA1234'}

> 这里有一个问题就是“今天”这个时间，模型根据自己的知识将“今天”变成了某一个具体日期，但是这个日期是不对的。这个在之后会解决。

### 整合

最后，我们把上面这些整合起来

In [47]:
destination_chains = {}
destination_chains["recommend"] = recommend_chain
destination_chains["faq"] = faq_chain
destination_chains["order_search"] = order_search_chain_seq
destination_chains["order_flight"] = order_flight_chain_seq

names_and_descriptions = [
    ("recommend", ["商品推荐"]),
    ("faq", ["问答-退换货及商品购买咨询"]),
    ("order_search", ["查询商品购买订单"]),
    ("order_flight", ["预定航班"])
]

from langchain.chains.router.embedding_router import EmbeddingRouterChain
from langchain.chains import ConversationChain
from langchain.chains.router import MultiPromptChain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='GanymedeNil/text2vec-large-chinese')

router_chain = EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions, FAISS, embeddings, routing_keys=["question"]
)
default_chain = ConversationChain(llm=alpaca_model, input_key output_key="text")

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: GanymedeNil/text2vec-large-chinese
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ValidationError: 12 validation errors for MultiPromptChain
destination_chains -> order_search -> prompt
  field required (type=value_error.missing)
destination_chains -> order_search -> llm
  field required (type=value_error.missing)
destination_chains -> order_search -> chains
  extra fields not permitted (type=value_error.extra)
destination_chains -> order_search -> input_variables
  extra fields not permitted (type=value_error.extra)
destination_chains -> order_search -> output_variables
  extra fields not permitted (type=value_error.extra)
destination_chains -> order_search -> return_all
  extra fields not permitted (type=value_error.extra)
destination_chains -> order_flight -> prompt
  field required (type=value_error.missing)
destination_chains -> order_flight -> llm
  field required (type=value_error.missing)
destination_chains -> order_flight -> chains
  extra fields not permitted (type=value_error.extra)
destination_chains -> order_flight -> input_variables
  extra fields not 

In [48]:
?EmbeddingRouterChain.from_names_and_descriptions

Signature:
EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions: 'Sequence[Tuple[str, Sequence[str]]]',
    vectorstore_cls: 'Type[VectorStore]',
    embeddings: 'Embeddings',
    **kwargs: 'Any',
) -> 'EmbeddingRouterChain'
Docstring: Convenience constructor.
File:      ~/anaconda3/envs/python3/lib/python3.10/site-packages/langchain/chains/router/embedding_router.py
Type:      method

#### 出错分析
可能是因为我们使用了 TransformationChain、SequentialChain等不同的Chain，他们内部会有不同的input、output，虽然我指定了 destination_chains 中每个Chain的输入都是 “question”，输出都是 “text”，但是还是提示 input_variables、output_variables相关的错误。

所以，我们还是手动的通过embedding识别意图。

In [49]:
names_and_descriptions = [
    ("recommend", ["商品推荐"]),
    ("faq", ["问答-退换货及商品购买咨询"]),
    ("order_search", ["查询商品购买订单"]),
    ("order_flight", ["预定航班"])
]

from langchain.docstore.document import Document
from langchain.vectorstores import FAISS

documents = []
for name, descriptions in names_and_descriptions:
    for description in descriptions:
        documents.append(
            Document(page_content=description, metadata={"name": name})
        )
vectorstore = FAISS.from_documents(documents, embeddings)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [53]:
intent = vectorstore.similarity_search("想要预定今天到上海的CA1234的航班", k=1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [60]:
func = intent[0].metadata["name"]
print(func)

order_flight


In [62]:
target_chain = destination_chains[func]

In [63]:
target_chain("想要预定今天到上海的CA1234的航班")

INFO:root:get_flight_func_inputs:{'question': '想要预定今天到上海的CA1234的航班'}
INFO:root:prompt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 从以下对输入中信息提取'['flight_date', 'flight_no']', 不要尝试回答问题, 不要编造答案, 不要使用已有知识, 找不到所需信息则返回空, 结果用json格式.### Input: 想要预定今天到上海的CA1234的航班 

### Response:

INFO:root:model_kwargs: {'temperature': 0.001, 'top_p': 0.3}




> Entering new SequentialChain chain...


INFO:root:response_json: {'answer': '[{"flight_date": "2021-08-22", "flight_no": "CA1234"} ]'}
INFO:root:==== search_order inputs:{'question': '想要预定今天到上海的CA1234的航班', 'input': ['flight_date', 'flight_no'], 'params': '[{"flight_date": "2021-08-22", "flight_no": "CA1234"} ]'}



> Finished chain.


{'question': '想要预定今天到上海的CA1234的航班', 'text': '预定：2021-08-22, 航班号：CA1234'}